# AP(P)3RO PROJECT #

### Projet de deneigement intelligent de Montréal, Canada ###

#### M. Le Gras, A. Naullet, A. Calixte, P. Beaunieux ####

In [14]:
import multiprocessing as mp

import numpy as np
import osmnx as ox
from copy import copy

## Partie 1: Cartographie de la ville par le drone ##

<h4>Rappel Objectif:</h4> 
Calcul du parcours de toute la ville par un drone.

Ceci s'apparente donc au calcul d'un chemin Eulerien du graph correspond a Montréal

In [15]:
place = "Montreal, Canada" 
G = ox.graph_from_place(place, network_type="walk")